<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/AdvML2024_ex02notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex02notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)

※ この notebook は，授業時間中の解説や板書と併用することを想定して作っていますので，説明が不十分なところが多々あります．


----
## 汎化と過適合
----

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

---
### 多項式回帰

「汎化と過適合」の説明で使う道具として，1入力1出力のデータに多項式を当てはめる **多項式回帰** を導入する．

**［多項式回帰（最小二乗法による多項式当てはめ）の問題設定］**

変数 $x$ と $y$ の値のペア $N$ 組から成るデータ
$$
(x_1, y_1), (x_2, y_2), \ldots, (x_N, y_N)
$$
がある．変数 $x$ の値から $y$ の値が決まるものとし，そのモデルとして $(D+1)$ 個のパラメータをもつ $D$ 次多項式

$$
y = f(x) = w_0 + w_1 x + w_2x^2 + \cdots + w_Dx^{D} \qquad (1)
$$

を考える．


このとき，モデルの出力 $f(x_n)$ とその正解の値 $y_n$ との間の二乗誤差の和
$$
\sum_{n=1}^{N}(y_n - f(x_n))^2
$$
を最小にするパラメータ $w_0, w_1, \ldots, w_D$ を求めたい．



この多項式回帰の問題は，線形回帰（平面当てはめ）の問題をちょこっと変形したものとみなせる．

平面当てはめでは，$1, x_1, x_2, \ldots, x_D$ という $(D+1)$ 個の値をならべた $(D+1)$ 次元ベクトル（$1\times (D+1)$ 行列）
$$
\mathbf{x} = (1, x_1, x_2, x_3, \ldots, x_D)
$$
を考えたが，そのかわりに
$$
\mathbf{x} = (1, x, x^2, x^3, \ldots, x^D)
$$
としてやれば ok．こうすれば，平面当てはめと同様に$(D+1)$個のパラメータをならべたベクトル（$1\times (D+1)$ 行列）を
$$
\mathbf{w} = (w_0, w_1, w_2, \ldots, w_D)
$$
として，式$(1)$を
$$
y = f(\mathbf{x}) = \mathbf{w}\cdot\mathbf{x} \qquad (2)
$$
と表せる．以下，平面当てはめの定式化と同じなので，正規方程式も同じ形になる．

つまり，$N\times(D+1)$ 行列 $X$ と $N\times 1$ 行列 $Y$ を

$$
X = \begin{pmatrix}
\mathbf{x}_1\\
\mathbf{x}_2\\
\vdots\\
\mathbf{x}_N\\
\end{pmatrix}
=
\begin{pmatrix}
1 & x_{1} & x_{1}^{2} & \cdots & x_{1}^{D}\\
1 & x_{2} & x_{2}^{2} & \cdots & x_{2}^{D}\\
& & \vdots\\
1 & x_{N} & x_{N}^{2} & \cdots & x_{N}^{D}\\
\end{pmatrix}
\qquad
Y = \begin{pmatrix}
y_1\\ y_2 \\ \vdots \\ y_N
\end{pmatrix}
$$

とおくと，二乗誤差を最小にするパラメータ $\mathbf{w}$ は次の正規方程式の解である．

$$
X^{\top}X\mathbf{w}^{\top}
= X^{\top}Y
$$


---
### 正弦波の多項式回帰の実験

#### 問題

$x$ と $y$ の間の真の関係が $y = h(x) = \sin{(2\pi x)} + 1$ という式で表されるときに，区間 $[0, 1]$ で一様分布する $x$ の値が $N$ 個得られ，それぞれの値に対応する $y$ の値も同数得られるとする．それらを $(x_n, y_n)$ ($n = 1, 2, \ldots, N$) とする．
ただし，得られる $y_n$ にはノイズが乗っており，$y_n = h(x_n) + \epsilon_n$ と表されるものとする．$\epsilon_n$ は標準偏差 $0.1$ の正規乱数である．

$(x_n, y_n)$  ($n = 1, 2, \ldots, N$) のみが与えられる（$h(x)$ の式やどんなノイズが乗っているかは未知）ときに，$x$ から $y$ の値を予測するモデルを作りたい．


In [ ]:
### データを生成する関数
#
def generateData(N, trueFunc=False, seed=None, sigma=0.0):

    if trueFunc:
        x = np.linspace(-0.1, 1.1, num=N)
        y = np.sin(2*np.pi*x) + 1
    else:
        # 乱数生成器
        rng = np.random.default_rng(seed)
        x = rng.uniform(0.0, 1.0, N)
        y = np.sin(2*np.pi*x) + 1 + sigma*rng.standard_normal(N)

    return x, y

In [ ]:
x_true, y_true  = generateData(100, trueFunc=True)
x_noisy, y_noisy = generateData(25, sigma=0.1)
fig, ax = plt.subplots()
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.2, 2.2)
ax.plot(x_true, y_true, color='blue', label='true function')
ax.scatter(x_noisy, y_noisy, color='red', label='noisy data')
ax.legend()
plt.show()

#### 多項式回帰をやってみよう

以下の「多項式次数」をいろいろ変えて実行してみよう．

In [ ]:
##### 1, x, x^2, x^3, ..., x^D をならべたデータ行列（N x (D+1)）をつくる
#
def makeDataMatrix(x, D):

    N = x.shape[0]
    X = np.zeros((N, D+1))
    X[:, 0] = 1
    for d in range(1, D+1):
        X[:, d] = x**d

    return X

In [ ]:
### 線形回帰
#
def LinearRegression(X, y):
    # 正規方程式の左辺の行列を A とする
    A = X.T @ X
    # 正規方程式の右辺のベクトルを b とする
    b = X.T @ y
    # np.linalg.solve を使って正規方程式を解き，解を w とする
    w = np.linalg.solve(A, b)
    return w

In [ ]:
# 実験条件
NL, NT = 10, 1000 # 学習データ数，テストデータ数
sig = 0.1 # ノイズの標準偏差
D = 1 # 多項式次数

# 学習データを作って多項式回帰
xL, yL = generateData(NL, sigma=sig)
XL = makeDataMatrix(xL, D)
w = LinearRegression(XL, yL)
yL_est = XL @ w
msqeL = np.mean((yL - yL_est)**2)

# テストデータを作って予測
xT, yT = generateData(NT, sigma=sig)
XT = makeDataMatrix(xT, D)
yT_est = XT @ w
msqeT = np.mean((yT - yT_est)**2)

## グラフを描く
#
x_true, y_true = generateData(1000, trueFunc=True)
X_true = makeDataMatrix(x_true, D)
y_est = X_true @ w
fig, ax = plt.subplots()
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.2, 2.2)
ax.plot(x_true, y_true, color='blue', label='true function')
ax.scatter(xL, yL, color='red', label='training data')
ax.plot(x_true, y_est, color='green', label='fitted curve')
ax.legend()
plt.show()

# 試行ごとの平均二乗誤差の平均を表示
print(f'D = {D}, msqeL = {msqeL:.5f}, msqeT = {msqeT:.5f}')

`msqeL` は，学習データに対する平均二乗誤差(mean squared error)である．モデルを $f(x)$ と表すとき，これは次式で与えられる．

$$
\frac{1}{N}\sum_{n=1}^{N}(y_n - f(x_n))^2
$$

`msqeT` は，学習データとは別に生成されたテストデータに対する平均二乗誤差である．ここでは，テストデータの生成条件は学習データと同じで，データ数 $N$ のみ異なっている．

観察しよう:
- 多項式の次数を変えると `msqeL` はどのように変化するか
- 同じく `msqeT` はどのように変化するか

次のセルを実行すると，上記の実験をもう少し大規模に行うことができる．ただし，学習データの数を $10$ から $25$ に増やしており，多項式次数 $D$ の値を一つ決めるごとに学習データ（とテストデータ）を1000通り作って1000通りのモデルを学習させている．

In [ ]:
# 実験条件
Dlist = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # 多項式次数
Ntrial = 1000 # 試行回数
NL, NT = 25, 1000 # 学習データ数，テストデータ数
sig = 0.1 # ノイズの標準偏差

msqeL = np.zeros((len(Dlist), Ntrial))
msqeT = np.zeros((len(Dlist), Ntrial))

for i, D in enumerate(Dlist):
    for n in range(Ntrial):
        xL, yL = generateData(NL, sigma=0.1, seed=2*n)
        XL = makeDataMatrix(xL, D)
        w = LinearRegression(XL, yL)
        yL_est = XL @ w
        msqeL[i, n] = np.mean((yL - yL_est)**2)
        xT, yT = generateData(NT, sigma=0.1, seed=2*n+1)
        XT = makeDataMatrix(xT, D)
        yT_est = XT @ w
        msqeT[i, n] = np.mean((yT - yT_est)**2)
    eL, eT = np.mean(msqeL[i, :]), np.mean(msqeT[i, :])
    print(f'# D = {D}, msqeL = {eL:.4g}, msqeT = {eT:.4g}')

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].boxplot(msqeL.T)
ax[0].set_xticklabels(Dlist)
ax[0].set_ylim(0, 0.1)
ax[0].set_xlabel('D')
ax[0].set_title('msqeL')
ax[1].boxplot(msqeT.T)
ax[1].set_xticklabels(Dlist)
ax[1].set_ylim(0, 0.1)
ax[1].set_xlabel('D')
ax[1].set_title('msqeT')
plt.show()

グラフの横軸は多項式次数 $D$ を表す．左図の縦軸は，1000回の試行ごとの学習データに対する平均二乗誤差を表し，右図の縦軸は，テストデータに対する同様の値を表す．箱ひげ図なので，の箱の中の赤線が1000試行の平均）．

観察しよう:
- 多項式の次数を変えると `msqeL` はどのように変化するか
- 同じく `msqeT` はどのように変化するか

---
### 汎化と過適合

データが少ないときに次数の大きい多項式を当てはめようとすると，学習データにはよく当てはまっても，その背後にある真の関数関係をうまくとらえられるとは限らず，未知のデータ（テストデータ）に対する当てはまりは逆に大きくなってしまうことがある．

**汎化** できない，**汎化能力** が低い，**過適合** (over-fitting, 過学習)している

一般に，パラメータ数の多い複雑な学習機械ほど学習データによく当てはまるが，過適合によって未知データにうまく汎化できなくなりがちである．
逆に言えば，複雑なモデルでもたくさんのデータを用いて学習させることができるならば，過適合は起こりにくくなる．



以下は，上記と同じ問題で，学習データ数を増やしてみた例である．

In [ ]:
# 実験条件
NL, NT = 100, 1000 # 学習データ数，テストデータ数
sig = 0.1 # ノイズの標準偏差
D = 9 # 多項式次数

# 学習データを作って多項式回帰
xL, yL = generateData(NL, sigma=sig)
XL = makeDataMatrix(xL, D)
w = LinearRegression(XL, yL)
yL_est = XL @ w
msqeL = np.mean((yL - yL_est)**2)

# テストデータを作って予測
xT, yT = generateData(NT, sigma=sig)
XT = makeDataMatrix(xT, D)
yT_est = XT @ w
msqeT = np.mean((yT - yT_est)**2)

## グラフを描く
#
x_true, y_true = generateData(1000, trueFunc=True)
X_true = makeDataMatrix(x_true, D)
y_est = X_true @ w
fig, ax = plt.subplots()
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.2, 2.2)
ax.plot(x_true, y_true, color='blue', label='true function')
ax.scatter(xL, yL, color='red', label='training data')
ax.plot(x_true, y_est, color='green', label='fitted curve')
ax.legend()
plt.show()

# 試行ごとの平均二乗誤差の平均を表示
print(f'D = {D}, msqeL = {msqeL:.5f}, msqeT = {msqeT:.5f}')

---
## 正則化とリッジ回帰
---

真の関数関係に合うようなモデルパラメータ数やモデルの複雑度があらかじめわかるならば，適切に設定して汎化性能の高いモデルが得られるかもしれない．しかし，そんなものは通常は分からない．どうしよう？


上の多項式回帰の実験で，得られたパラメータの値を表示させてみると，次のようになっている．

In [ ]:
# 実験条件
NL, NT = 10, 1000 # 学習データ数，テストデータ数
sig = 0.1 # ノイズの標準偏差
Dlist = [1, 3, 5, 7, 9] # 多項式次数

for D in Dlist:
    xL, yL = generateData(NL, sigma=sig)
    XL = makeDataMatrix(xL, D)
    w = LinearRegression(XL, yL)
    print(f'D = {D}, w = {w}, ||w||^2 = {w @ w}')

各行の末尾には，パラメータの $L_2$ ノルム，すなわちパラメータの値の二乗和

$$
||\mathbf{w}||^2 = w_0^2 + w_1^2 + \ldots + w_D^2
$$

の値も表示してある．

次数の大きい（パラメータ数の多い）モデルでは，パラメータの値が大きくなりがち．そうならないように制約したらどうだろう？

---
### リッジ回帰



#### リッジ回帰とは

線形回帰（最小二乗法による平面当てはめ）の問題において，

$$
E(\mathbf{w}) = \frac{1}{2}\sum_{n=1}^N (y_n - \mathbf{w}\cdot\mathbf{x}_n)^2$$

のかわりに

$$
F(\mathbf{w}) = E(\mathbf{w}) + \frac{\alpha}{2}||\mathbf{w}||^2
$$

を最小化する．$\alpha$ は $0$ 以上の定数とする．右辺第2項は，正則化項（$L_2$正則化項）と呼ばれる．二乗誤差に正則化項を加えた誤差関数を最小化する線形回帰を，
**リッジ回帰** (Ridge Regression) または $L_2$正則化付き線形回帰という．

パラメータの $L_2$ ノルムが過大にならないように学習させることで，モデルの実質的な自由度が抑えられ，過適合が抑制されることが期待できる．


#### 正規方程式の導出

$F(\mathbf{w})$ を最小にするパラメータ $\mathbf{w}$ は次の正規方程式の解である．

$$
\mathbf{w}\left( X^{\top}X + \alpha I \right)
= Y^{\top}X
$$

両辺を転置して書くと次のようになる．

$$
\left( X^{\top}X + \alpha I \right)\mathbf{w}^{\top}
= X^{\top}Y
$$


### （よだんだよん）正則化という言葉の意味

行列 $A$ を $A = X^{\top}X$ とおく．$A^{\top} = \left(X^{\top}X\right)^{\top} = X^{\top}\left(X^{\top}\right)^{\top}  = X^{\top}X = A$ より，$A$ が対称行列であることは明らか．対称行列の固有値は実数であり，固有ベクトルは互いに直交するようにとることができる．証明は省略するが，行列 $A$ は「準正定値」対称行列（固有値が $0$ 以上）となる．

一方，任意の正方行列 $A$ の固有値を $\lambda$，対応する固有ベクトルを（列ベクトルとして） $\mathbf{u}$ とおくと，$A\mathbf{u} = \lambda\mathbf{u}$ である．このとき，$B =A + \alpha I$ に対して $B\mathbf{u} = A\mathbf{u} + \alpha\mathbf{u} = (\lambda + \alpha)\mathbf{u}$ が成り立つ．したがって，$\lambda + \alpha$ が $B$ の固有値である．

以上をあわせて考えると，パラメータ数に対してデータ数が少なくて $X^{\top}X$ のランクが落ちている = 正則でない = いくつかの固有値が $0$ である，という場合でも，$X^{\top}X + \alpha I$ の固有値は $\alpha$ 以上になる．したがって，$\alpha > 0$ とすればこの行列は正則となる．

---
### リッジ回帰の実験



正弦波の多項式回帰の例でリッジ回帰をやってみよう．

※注: ここでは，モデルのパラメータのうち，定数項を表す $w_0$ は正則化の対象外とするようにしています．線形回帰のモデルでは定数項には制約を加えない方が適切なことが多いので．

In [ ]:
### リッジ回帰
#
def RidgeRegression(X, y, lam, excludeBias=True):
    # 正規方程式の左辺の行列を A とする
    eyet = np.ones(X.shape[1])
    if excludeBias:
        eyet[0] = 0.0
    A = X.T @ X + lam * np.diag(eyet)
    # 正規方程式の右辺のベクトルを XTY とする
    b = X.T @ y
    #print(X.shape, A.shape, b.shape)
    # np.linalg.solve を使って正規方程式を解き，解を w とする
    w = np.linalg.solve(A, b)
    return w

次のセルの変数 `alpha` の値が，正則化項の係数 $\alpha$ を表している．適当に変えて結果を観察しよう．

In [ ]:
# 実験条件
NL, NT = 25, 1000 # 学習データ数，テストデータ数
sig = 0.1 # ノイズの標準偏差
D = 9 # 多項式次数
alpha = 0.0 # 0.0, 0.0001, 0.001, 0.01, 0.1, 1.0

# 学習データを作って多項式回帰
xL, yL = generateData(NL, sigma=sig)
XL = makeDataMatrix(xL, D)
w = RidgeRegression(XL, yL, alpha)
yL_est = XL @ w
msqeL = np.mean((yL - yL_est)**2)

# テストデータを作って予測
xT, yT = generateData(NT, sigma=sig)
XT = makeDataMatrix(xT, D)
yT_est = XT @ w
msqeT = np.mean((yT - yT_est)**2)

## グラフを描く
#
x_true, y_true = generateData(1000, trueFunc=True)
X_true = makeDataMatrix(x_true, D)
y_est = X_true @ w
fig, ax = plt.subplots()
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.2, 2.2)
ax.plot(x_true, y_true, color='blue', label='true function')
ax.scatter(xL, yL, color='red', label='training data')
ax.plot(x_true, y_est, color='green', label='fitted curve')
ax.legend()
plt.show()

# 試行ごとの平均二乗誤差の平均を表示
print(f'D = {D}, alpha = {alpha}, msqeL = {msqeL:.5f}, msqeT = {msqeT:.5f}')

In [ ]:
# 実験条件
D = 9
lamlist = [0.0, 0.00001, 0.0001, 0.001, 0.01] # 正則化パラメータ
Ntrial = 1000 # 試行回数
NL, NT = 25, 1000 # 学習データ数，テストデータ数
sig = 0.1 # ノイズの標準偏差

msqeL = np.zeros((len(lamlist), Ntrial))
msqeT = np.zeros((len(lamlist), Ntrial))

for i, lam in enumerate(lamlist):
    for n in range(Ntrial):
        xL, yL = generateData(NL, sigma=0.1, seed=2*n)
        XL = makeDataMatrix(xL, D)
        w = RidgeRegression(XL, yL, lam)
        yL_est = XL @ w
        msqeL[i, n] = np.mean((yL - yL_est)**2)
        xT, yT = generateData(NT, sigma=0.1, seed=2*n+1)
        XT = makeDataMatrix(xT, D)
        yT_est = XT @ w
        msqeT[i, n] = np.mean((yT - yT_est)**2)
    eL, eT = np.mean(msqeL[i, :]), np.mean(msqeT[i, :])
    print(f'# D = {D}, lam = {lam}, msqeL = {eL:.4g}, msqeT = {eT:.4g}')

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].boxplot(msqeL.T)
ax[0].set_xticklabels(lamlist)
ax[0].set_ylim(0, 0.1)
ax[0].set_xlabel(r'$\alpha$')
ax[0].set_title('msqeL')
ax[1].boxplot(msqeT.T)
ax[1].set_xticklabels(lamlist)
ax[1].set_ylim(0, 0.1)
ax[1].set_xlabel(r'$\alpha$')
ax[1].set_title('msqeT')
plt.show()